### The Cross-Correlation Operation

In [1]:
import tensorflow as tf


def corr2d(X, K):  #@save
    """Compute 2D cross-correlation."""
    h, w = K.shape
    Y = tf.Variable(tf.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1)))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j].assign(tf.reduce_sum(
                X[i: i + h, j: j + w] * K))
    return Y

In [2]:
X = tf.constant([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
K = tf.constant([[0.0, 1.0], [2.0, 3.0]])
corr2d(X, K)

<tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[19., 25.],
       [37., 43.]], dtype=float32)>

### Convolutional Layers

In [3]:
class Conv2D(tf.keras.layers.Layer):
    def __init__(self):
        super().__init__()

    def build(self, kernel_size):
        initializer = tf.random_normal_initializer()
        self.weight = self.add_weight(name='w', shape=kernel_size,
                                      initializer=initializer)
        self.bias = self.add_weight(name='b', shape=(1, ),
                                    initializer=initializer)

    def call(self, inputs):
        return corr2d(inputs, self.weight) + self.bias

### Object Edge Detection in Images

The middle four columns are black (0) and the rest are white (1)

In [4]:
X = tf.Variable(tf.ones((6, 8)))
X[:, 2:6].assign(tf.zeros(X[:, 2:6].shape))
X

<tf.Variable 'Variable:0' shape=(6, 8) dtype=float32, numpy=
array([[1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.],
       [1., 1., 0., 0., 0., 0., 1., 1.]], dtype=float32)>

#### Kernel

In [5]:
K = tf.constant([[1.0, -1.0]])
K

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 1., -1.]], dtype=float32)>

#### We are ready to perform the cross-correlation operation with arguments X (our input) and K (our kernel). As you can see, we detect 1 for the edge from white to black and -1 for the edge from black to white. All other outputs take value 0.

In [6]:
Y = corr2d(X, K)
Y

<tf.Variable 'Variable:0' shape=(6, 7) dtype=float32, numpy=
array([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  1.,  0.,  0.,  0., -1.,  0.]], dtype=float32)>

#### We can now apply the kernel to the transposed image. As expected, it vanishes. The kernel K only detects vertical edges.

In [7]:
corr2d(tf.transpose(X), K)

<tf.Variable 'Variable:0' shape=(8, 5) dtype=float32, numpy=
array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]], dtype=float32)>

### Learning a Kernel

#### Model

In [8]:
# Construct a two-dimensional convolutional layer with 1 output channel and a
# kernel of shape (1, 2). For the sake of simplicity, we ignore the bias here
conv2d = tf.keras.layers.Conv2D(1, (1, 2), use_bias=False)

#### Training

In [9]:
num_epochs = 10
X = tf.reshape(X, (1, 6, 8, 1))
y = tf.reshape(Y, (1, 6, 7, 1))
lr = 3e-2  # Learning rate

y_hat = conv2d(X)
for epoch in range(num_epochs):
    # Compute gradients and update parameters
    with tf.GradientTape(watch_accessed_variables=False) as tape:
        tape.watch(conv2d.weights[0])
        y_hat = conv2d(X)
        l = (abs(y_hat - y)) ** 2
        # Update the kernel
        grads = tape.gradient(l, conv2d.weights[0])
        weights = conv2d.get_weights()
        weights[0] = conv2d.weights[0] - tf.multiply(lr, grads)
        conv2d.set_weights(weights)
        print(f'epoch {epoch + 1}, loss {tf.reduce_sum(l):.3f}')

epoch 1, loss 19.268
epoch 2, loss 9.164
epoch 3, loss 4.568
epoch 4, loss 2.392
epoch 5, loss 1.313
epoch 6, loss 0.751
epoch 7, loss 0.444
epoch 8, loss 0.269
epoch 9, loss 0.166
epoch 10, loss 0.104


#### Now we will take a look at the kernel tensor we learned.

In [10]:
tf.reshape(conv2d.get_weights()[0], (1, 2))

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[ 0.95508915, -1.0202311 ]], dtype=float32)>

### Cross-Correlation and Convolution

What we did is not strict convolution but cross-correlation. A strict convolution can be done by doing transformation on the kernel and since, the kernel is learned, so it doesn't matter if we do a strict convolution, so we're good with cross-correlation and we refer to it as convolution, even though that's not correct.

### Feature Map and Receptive Field

The output of cross-correlation operation is called Feature map. The Feature Map becomes an input for the subsequent layers, given x as input to any subsequent layer, then Receptive Fields refer to all the elements that contribute to calculate x.